In [1]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu
from src.me_to_neurazi import me_to_neurazi

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

In [2]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(~pl.col('rok').is_null()).sort(by='rok')
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null())
print(len(df))

1001279
991612


In [3]:
df = df.filter(pl.col("rok") >= 1800)

In [4]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","041.parquet")), left_on="001", right_on="001", how="left")

In [5]:
predfiltr = df.filter(pl.col('stran') >= 30).unique(subset=['rok','245_a','100_a'])

In [6]:
preklady = predfiltr.drop_nulls(subset=["041_h","rok"]).explode("041_h").filter(pl.col("041_h").is_not_null()).filter(pl.col("041_h") != "cze").unique(subset=['rok','245_a','100_a'])

In [7]:
preklady_celkem = preklady.group_by(['rok']).len().rename({'len':'prekladu_celkem'}).sort(by="rok")
preklady_celkem

rok,prekladu_celkem
i64,u32
1801,4
1802,5
1803,6
1804,9
1805,2
1806,3
1807,4
1808,6
1809,7


## Pokus č. 2, hezčejší

In [9]:
nahradit = {
    'eng': 'angličtina', 'ger': 'němčina', 'slo': 'slovenština', 'rus': 'ruština', 'fre': 'francouzština', 'lat': 'latina', 'pol': 'polština','ita':'italština','ostatní cizí jazyky':'ostatní cizí jazyky'
}

In [10]:
historicky_vyvoj = preklady.group_by(
    ["rok","041_h"]
).len().sort(by=['rok','len'],descending=[False,True]).unique(
    subset=['rok'],keep="first"
).join(
    preklady_celkem, on='rok', how='left'
).with_columns((pl.col('prekladu_celkem') - pl.col('len')).alias('ostatní jazyky')).rename(
    {'len':'nejzastoupenější jazyk'}
).join(predfiltr.group_by('rok').len().rename({'len':'všechny'}), on='rok', how='left').with_columns(
    pl.col('nejzastoupenější jazyk') / pl.col('všechny')
).with_columns(
    pl.col('ostatní jazyky') / pl.col('všechny')
).with_columns(
    (pl.col("prekladu_celkem") / pl.col("všechny")).alias("všechny překlady")
)
#.with_columns(
  #  pl.col('041_h').replace_strict(nahradit)
#)

historicky_vyvoj.filter(pl.col('prekladu_celkem') < 80).tail()

rok,041_h,nejzastoupenější jazyk,prekladu_celkem,ostatní jazyky,všechny,všechny překlady
i64,str,f64,u32,f64,u32,f64
1870,"""ger""",0.139535,75,0.109635,301,0.249169
1871,"""ger""",0.109023,68,0.146617,266,0.255639
1877,"""ger""",0.098214,77,0.130952,336,0.229167
1878,"""ger""",0.101124,67,0.087079,356,0.188202
1879,"""ger""",0.076023,68,0.122807,342,0.19883


In [11]:
historicky_vyvoj.filter(pl.col("rok").is_between(1935,1950))

rok,041_h,nejzastoupenější jazyk,prekladu_celkem,ostatní jazyky,všechny,všechny překlady
i64,str,f64,u32,f64,u32,f64
1935,"""eng""",0.045066,501,0.102504,3395,0.14757
1936,"""eng""",0.051386,641,0.111676,3931,0.163063
1937,"""eng""",0.048477,663,0.119797,3940,0.168274
1938,"""eng""",0.052239,478,0.08496,3484,0.137199
1939,"""eng""",0.051177,390,0.094509,2677,0.145685
1940,"""ger""",0.053405,505,0.089291,3539,0.142696
1941,"""ger""",0.076259,531,0.081918,3357,0.158177
1942,"""ger""",0.123358,350,0.076528,1751,0.199886
1943,"""ger""",0.130137,218,0.035769,1314,0.165906


In [58]:
jak_rikame_ostatnim = 'celkový podíl překladů'

In [60]:
vyvoj_top = historicky_vyvoj.select(pl.col(["rok","041_h","nejzastoupenější jazyk"])).rename({"nejzastoupenější jazyk":"podil"}).sort(by="rok").with_columns(pl.col("041_h").replace(nahradit)) # .with_columns(pl.col("podil").rolling_mean(window_size=2))
vyvoj_rest = historicky_vyvoj.select(pl.col(['rok','všechny překlady'])).with_columns(pl.lit('všechny překlady').alias("041_h")).rename({"všechny překlady":"podil"}).with_columns(pl.col("041_h").replace({'všechny překlady': jak_rikame_ostatnim})).sort(by="rok").with_columns(pl.col("041_h").replace(nahradit)) # .with_columns(pl.col("podil").rolling_mean(window_size=2))

In [76]:
gr_top = alt.Chart(alt_friendly(vyvoj_top.filter(pl.col("rok").is_between(1900,2024))),
                  title=["Podíl překladů na knižní produkci a nejzastoupenější původní jazyky"]).mark_bar(width=1.5).encode(
        alt.X('rok:T', title=None),
    alt.Y('podil:Q', title=None, axis=alt.Axis(labelExpr="datum.label * 100 + ' %'", orient='right', domainOpacity=0, tickColor='#DCDDD6')),
        alt.Color("041_h:N", 
                  scale=alt.Scale(range=['#445B78', '#81A9D5', '#EED801', '#D6534B', '#F2E9DE']),
                  legend=alt.Legend(orient="top",title=None,columns=5),
                  sort=['angličtina','francouzština','němčina','ruština',jak_rikame_ostatnim])
)

gr_rest = alt.Chart(alt_friendly(vyvoj_rest.filter(pl.col("rok").is_between(1900,2024)))).mark_bar(width=2).encode(
        alt.X('rok:T', title=None, axis=alt.Axis(tickCount=10)),
    alt.Y('podil:Q', title=None),
    alt.Color("041_h:N", scale=alt.Scale(), sort=['angličtina','francouzština','němčina','ruština',jak_rikame_ostatnim])
)

vyvoj_prekladu = alt.layer(gr_rest, gr_top).properties(
    width=kredity['sirka'] * 1.2, height=kredity['vyska_nizkych'] * 1.66, 
    autosize={'type': 'fit-x', 'contains': 'padding'}).configure_view(stroke='transparent').configure_axisX(grid=False, domain=True)

vyvoj_prekladu

alt.LayerChart(...)

In [78]:
me_to_neurazi(vyvoj_prekladu, soubor="01_vyvoj_prekladu", kredity=kredity['default'])

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/01_vyvoj_prekladu.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/01_vyvoj_prekladu.svg" width="100%" alt="Omlouváme se, ale alternativní text se nepodařilo vygenerovat. Texty v grafu by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [15]:
preklady.filter((pl.col('041_h') == 'ger') & (pl.col('rok') == 1993))

leader,001,100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,008,020_q,020_c,020_a,020_z,022_a,022_y,022_z,022_ind1,022_l,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,300_a,300_b,300_c,300_e,300_f,300_3,…,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_m,700_k,700_n,700_r,700_p,700_o,700_s,700_j,700_x,700_e,700_f,700_5,700_9,700_g,rok,stran,vazba,041_ind1,041_a,041_h,041_b,041_k,041_g,041_f,041_d,041_e,041_j,041_n,041_m
str,str,str,str,str,list[str],str,str,list[str],str,str,str,str,list[str],list[str],list[str],str,list[str],list[str],str,str,str,str,str,str,str,list[str],str,str,str,str,list[str],list[str],list[str],list[str],str,str,…,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str,str,list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str
""" nam a22 4500""","""np9409622""","""1""","""Frey, Britta""","""ola2002105888""","[""aut""]",null,null,null,null,null,"""940623s1993 xr u0…","""(brož.) :""","[""Kč 9,90""]","[""80-237-0167-3""]",null,null,null,null,null,null,"""1""","""0""","""Otec pro Sebastiana""",null,"""Britta Freyová ; Z něm. přel. …",null,null,null,null,null,"[""57 s. ;""]",null,"[""21 cm""]",null,null,null,…,"[""jx20040216029""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1993,57,"""brožovaná""","""1""","[""cze""]","""ger""",null,null,null,null,null,null,null,null,null
""" nam a22 4500""","""np9316145""","""1""","""Sark, Sylvia""",null,"[""aut""]",null,null,null,null,null,"""940120s1993 xr u0…","""(brož.) :""","[""Kč 19,90""]","[""80-237-0651-9""]",null,null,null,null,null,null,"""1""","""0""","""Cypřiše nad mořem""",null,"""Sylvia Sarková ; Z něm. přel. …",null,null,null,null,null,"[""125 s. ;""]",null,"[""19 cm""]",null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1993,125,"""brožovaná""","""1""","[""cze""]","""ger""",null,null,null,null,null,null,null,null,null
""" nam a22 4500""","""np9311122""","""1""","""Scott, Jennifer""","""jx20100610015""","[""aut""]",null,null,null,null,null,"""940120s1993 xr u0…","""(brož.) :""","[""Kč 19,90""]","[""80-237-0342-0""]",null,null,null,null,null,null,"""1""","""0""","""Pod karibským sluncem""",null,"""Jennifer Scottová ; [Z něm.] p…",null,null,null,null,null,"[""127 s. ;""]",null,"[""19 cm""]",null,null,null,…,"[""jx20040726011""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1993,127,"""brožovaná""","""1""","[""cze""]","""ger""",null,null,null,null,null,null,null,null,null
""" nam a22 4500""","""np9314804""","""1""","""Canter, Cal,""","""jx20100521010""","[""aut""]","""1921-1999""",null,null,null,null,"""940120s1993 xr u0…","""(brož.) :""","[""Kč 9,90""]","[""80-7116-570-0""]",null,null,null,null,null,null,"""1""","""0""","""Zelené kapky""",null,"""C. Canter ; z něm. přel. Jaros…",null,null,null,null,null,"[""60 s. ;""]",null,"[""20 cm""]",null,null,null,…,"[""jn20000620195""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1993,60,"""brožovaná""","""1""","[""cze""]","""ger""",null,null,null,null,null,null,null,null,null
""" nam a22 4500""","""np9409725""","""0""","""Bianca-Maria""","""mzk2003195018""","[""aut""]",null,null,null,null,null,"""940623s1993 xr u0…","""(brož.) :""","[""Kč 9,90""]","[""80-237-0265-3""]",null,null,null,null,null,null,"""1""","""0""","""Trápení""",null,"""Bianca-Maria ; Z něm. přel. Ji…",null,null,null,null,null,"[""62 s. ;""]",null,"[""20 cm""]",null,null,null,…,"[""jx20040721038""]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1993,62,"""brožovaná""","""1""","[""cze""]","""ger""",nu

## Pokus č. 1, to jsem to ještě neuměl tak báječně

In [17]:
nahradit = {
    'eng': 'angličtina', 'ger': 'němčina', 'slo': 'slovenština', 'rus': 'ruština', 'fre': 'francouzština', 'lat': 'latina', 'pol': 'polština','ita':'italština'
}

In [18]:
nejzastoupenejsi_jazyky = preklady.group_by('041_h').len().sort('len', descending=True)
nejzastoupenejsi_jazyky = nejzastoupenejsi_jazyky.get_column('041_h').to_list()
nejzastoupenejsi_jazyky = nejzastoupenejsi_jazyky[0:7]
print(nejzastoupenejsi_jazyky)
nejzastoupenejsi_jazyky = ['eng','ger','rus','fre','slo']
preklady_po_jazycich = preklady.filter(pl.col("rok").is_between(1900,2020)).group_by(['rok','041_h']).len().rename({'len':'prekladu'})
preklady_po_jazycich = preklady_po_jazycich.join(preklady_celkem, on='rok', how='full')
preklady_po_jazycich = preklady_po_jazycich.with_columns((pl.col('prekladu') / pl.col('prekladu_celkem')).alias('podíl'))
preklady_po_jazycich = preklady_po_jazycich.with_columns(pl.col('041_h').cast(str))
preklady_po_jazycich = preklady_po_jazycich.filter(pl.col("041_h").is_in(nejzastoupenejsi_jazyky)).sort('rok').rename({'041_h':'jazyk'}).with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
preklady_po_jazycich = preklady_po_jazycich.with_columns(pl.col('jazyk').replace_strict(nahradit))
preklady_po_jazycich

['eng', 'ger', 'rus', 'fre', 'pol', 'slo', 'ita']


rok,jazyk,prekladu,rok_right,prekladu_celkem,podíl
datetime[μs],str,u32,i64,u32,f64
1900-01-01 00:00:00,"""němčina""",32,1900,196,0.163265
1900-01-01 00:00:00,"""ruština""",20,1900,196,0.102041
1900-01-01 00:00:00,"""francouzština""",46,1900,196,0.234694
1900-01-01 00:00:00,"""angličtina""",42,1900,196,0.214286
1901-01-01 00:00:00,"""slovenština""",1,1901,240,0.004167
1901-01-01 00:00:00,"""němčina""",29,1901,240,0.120833
1901-01-01 00:00:00,"""angličtina""",38,1901,240,0.158333
1901-01-01 00:00:00,"""ruština""",22,1901,240,0.091667
1901-01-01 00:00:00,"""francouzština""",43,1901,240,0.179167


In [19]:
titulek = "Ze kterých jazyků se kdy překládaly knihy do češtiny"
podtitulek = ["Podíl jazyků na překladové literatuře – krásné, naučné i odborné."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [20]:
podily_faceted = alt.Chart(preklady_po_jazycich.to_pandas(), title={'text': titulek}, width=300).mark_area().encode(
    alt.X("rok:T", 
          title=None,
          axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
    ),
    alt.Y('podíl:Q', 
          axis=alt.Axis(labelExpr="datum.label * 100 + ' %'", orient='right', domainOpacity=0, tickColor='#DCDDD6'), 
          title=None
    ),
    alt.Color("jazyk:N", 
              scale=alt.Scale(range=['#687fa9','#84c0e4','#e3d83b','#b45058','#789256','#E0DAB5',]), ## '# ['#a7ddd6','#001f80','#e6e6e4','#e5db00','#e49996','#9cc002']), 
              title=None, 
              legend=None
    ),
    row=alt.Row(
        "jazyk:N", 
        title=None, 
        spacing=15, 
        header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='middle', labelFont='Asap')
    )
).resolve_axis(
    x='independent'
).properties(height=60, width=300).configure_view(stroke='transparent')

podily_faceted

alt.Chart(...)

## Počty jednotlivých jazyků

In [22]:
rocni_pocty = preklady.filter(pl.col("rok") >= 1800).group_by("rok").agg(pl.col("041_h").unique().len()).sort(by="rok")
rocni_pocty

rok,041_h
i64,u32
1801,1
1802,3
1803,2
1804,3
1805,2
1806,3
1807,2
1808,3
1809,3


In [23]:
alt_friendly(rocni_pocty)

,rok,041_h
0,1801-01-01,1
1,1802-01-01,3
2,1803-01-01,2
3,1804-01-01,3
4,1805-01-01,2
...,...,...
220,2021-01-01,56
221,2022-01-01,54
222,2023-01-01,51
223,2024-01-01,54


In [24]:
alt.Chart(alt_friendly(rocni_pocty), width=300,
         title=["Z kolika jednotlivých jazyků se překládaly knihy"]
         ).mark_area().encode(
    alt.X("rok:T", axis=alt.Axis(domainOpacity=0, tickCount=4, tickColor='#DCDDD6'), title=None),
    alt.Y("041_h", axis=alt.Axis(orient='right', tickCount=4, domainOpacity=0, tickColor='#DCDDD6'), title=None)
)

alt.Chart(...)